# 生成Apple PodCast字幕 / Apple PodCast Transcription with OpenAI's Whisper



In [1]:
#@markdown # **Check GPU type** 🕵️

#@markdown The type of GPU you get assigned in your Colab session defined the speed at which the video will be transcribed.
#@markdown The higher the number of floating point operations per second (FLOPS), the faster the transcription.
#@markdown But even the least powerful GPU available in Colab is able to run any Whisper model.
#@markdown Make sure you've selected `GPU` as hardware accelerator for the Notebook (Runtime → Change runtime type → Hardware accelerator).

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---
#@markdown **Factory reset your Notebook's runtime if you want to get assigned a new GPU.**

!nvidia-smi -L

!nvidia-smi
     

GPU 0: Tesla T4 (UUID: GPU-2bba1763-1e76-7de5-21dd-28696164c1f1)
Sat Mar 25 18:14:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                       

In [2]:
#@markdown **配置Whisper/Setup Whisper** 🏗️

!pip install requests beautifulsoup4
! pip install git+https://github.com/openai/whisper.git

import torch
import sys

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

print('Whisper installed，please execute next cell')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5o9tc7ht
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5o9tc7ht
  Resolved https://github.com/openai/whisper.git to commit 6dea21fd7f7253bfe450f1e2512a0fe47ee2d258
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 KB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=bb5c1d6fa52a76488f8159a7eece64af95ff7faa563f141ed605b224935fb3b2
  Stored in di

Using device: cuda:0


In [3]:
#@markdown # **Model selection** 🧠

#@markdown As of the first public release, there are 4 pre-trained options to play with:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
#@markdown | large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

#@markdown ---

Model = 'large-v2' #@param ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large', 'large-v2']
#@markdown ---
#@markdown **Run this cell again if you change the model.**

import whisper
from IPython.display import Markdown

whisper_model = whisper.load_model(Model)

if Model in whisper.available_models():
    display(Markdown(
        f"**{Model} model is selected.**"
    ))
else:
    display(Markdown(
        f"**{Model} model is no longer available.** Please select one of the following: - {' - '.join(whisper.available_models())}"
    ))

100%|██████████████████████████████████████| 2.87G/2.87G [00:12<00:00, 256MiB/s]


**large-v2 model is selected.**

In [26]:
#@markdown # **Apple Podcast selection** 🎙️

#@markdown Enter the URL of the Apple Podcast you want to transcribe.

#@markdown ---
#@markdown #### **Apple Podcast**
URL = "https://podcasts.apple.com/us/podcast/%E4%B9%B1%E7%BF%BB%E4%B9%A6/id1591595410?i=1000605159653" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the video.**

import os
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urlparse


def find_audio_url(html: str) -> str:
    # Find all .mp3 and .m4a URLs in the HTML content
    audio_urls = re.findall(r'https://[^\s^"]+(?:\.mp3|\.m4a)', html)

    # If there's at least one URL, return the first one
    if audio_urls:
        pattern = r'=https?://[^\s^"]+(?:\.mp3|\.m4a)'
        result = re.findall(pattern, audio_urls[-1])
        if result:
          return result[-1][1:]
        else:
          return audio_urls[-1]

    # Otherwise, return None
    return None

def get_file_extension(url: str) -> str:
    # Parse the URL to get the path
    parsed_url = urlparse(url)
    path = parsed_url.path

    # Extract the file extension using os.path.splitext
    _, file_extension = os.path.splitext(path)

    print("url", url, path, file_extension)
    # Return the file extension
    return file_extension

def download_apple_podcast(url: str, output_folder: str = 'downloads'):
    response = requests.get(url)
    if response.status_code != 200:
        print(
            f"Error: Unable to fetch the podcast page. Status code: {response.status_code}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    audio_url = find_audio_url(response.text)

    if not audio_url:
        print("Error: Unable to find the podcast audio url.")
        return

    episode_title = soup.find('span', {'class': 'product-header__title'})

    if not episode_title:
        print("Error: Unable to find the podcast title.")
        return

    episode_title = episode_title.text.strip().replace('/', '-')

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_file = os.path.join(output_folder, f"{episode_title}{get_file_extension(audio_url)}")

    with requests.get(audio_url, stream=True) as r:
        r.raise_for_status()
        with open(output_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)


    return episode_title, output_file


result = download_apple_podcast(URL)
if not result:
  print("Error: Unable to download podcast.")
else:
  (title, filepath) = result
  print(f"Downloaded podcast episode '{title}' to '{filepath}'")

url https://dts-api.xiaoyuzhoufm.com/track/61358d971c5d56efe5bcb5d2/64199cbe73768bea3509176a/media.xyzcdn.net/lt5grLkHEoR-ckvLwPJVNXCfTxzr.m4a /track/61358d971c5d56efe5bcb5d2/64199cbe73768bea3509176a/media.xyzcdn.net/lt5grLkHEoR-ckvLwPJVNXCfTxzr.m4a .m4a
Downloaded podcast episode '108.跟傅盛和王俊煜聊：大模型、产品经理和热门AI应‪用‬' to 'downloads/108.跟傅盛和王俊煜聊：大模型、产品经理和热门AI应‪用‬.m4a'


In [28]:
#@markdown # **Run the model** 🚀

#@markdown Run this cell to execute the transcription of the video. This can take a while and very based on the length of the video and the number of parameters of the model selected above.

#@markdown ## **Parameters** ⚙️

#@markdown ### **Behavior control**
#@markdown ---
language = "Auto detection"  # @param ['Auto detection', 'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Azerbaijani', 'Bashkir', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Breton', 'Bulgarian', 'Burmese', 'Castilian', 'Catalan', 'Chinese', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Estonian', 'Faroese', 'Finnish', 'Flemish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Lao', 'Latin', 'Latvian', 'Letzeburgesch', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Moldavian', 'Moldovan', 'Mongolian', 'Myanmar', 'Nepali', 'Norwegian', 'Nynorsk', 'Occitan', 'Panjabi', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Pushto', 'Romanian', 'Russian', 'Sanskrit', 'Serbian', 'Shona', 'Sindhi', 'Sinhala', 'Sinhalese', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tibetan', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uzbek', 'Valencian', 'Vietnamese', 'Welsh', 'Yiddish', 'Yoruba']
#@markdown > Language spoken in the audio, use `Auto detection` to let Whisper detect the language.
#@markdown ---
verbose = 'Live transcription' #@param ['Live transcription', 'Progress bar', 'None']
#@markdown > Whether to print out the progress and debug messages.
#@markdown ---
output_format = 'srt' #@param ['txt', 'vtt', 'srt', 'tsv', 'json', 'all']
#@markdown > Type of file to generate to record the transcription.
#@markdown ---
task = 'transcribe' #@param ['transcribe', 'translate']
#@markdown > Whether to perform X->X speech recognition (`transcribe`) or X->English translation (`translate`).
#@markdown ---

#@markdown 

#@markdown ### **Optional: Fine tunning** 
#@markdown ---
temperature = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown > Temperature to use for sampling.
#@markdown ---
temperature_increment_on_fallback = 0.2 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown > Temperature to increase when falling back when the decoding fails to meet either of the thresholds below.
#@markdown ---
best_of = 5 #@param {type:"integer"}
#@markdown > Number of candidates when sampling with non-zero temperature.
#@markdown ---
beam_size = 8 #@param {type:"integer"}
#@markdown > Number of beams in beam search, only applicable when temperature is zero.
#@markdown ---
patience = 1.0 #@param {type:"number"}
#@markdown > Optional patience value to use in beam decoding, as in [*Beam Decoding with Controlled Patience*](https://arxiv.org/abs/2204.05424), the default (1.0) is equivalent to conventional beam search.
#@markdown ---
length_penalty = -0.05 #@param {type:"slider", min:-0.05, max:1, step:0.05}
#@markdown > Optional token length penalty coefficient (alpha) as in [*Google's Neural Machine Translation System*](https://arxiv.org/abs/1609.08144), set to negative value to uses simple length normalization.
#@markdown ---
suppress_tokens = "-1" #@param {type:"string"}
#@markdown > Comma-separated list of token ids to suppress during sampling; '-1' will suppress most special characters except common punctuations.
#@markdown ---
initial_prompt = "" #@param {type:"string"}
#@markdown > Optional text to provide as a prompt for the first window.
#@markdown ---
condition_on_previous_text = True #@param {type:"boolean"}
#@markdown > if True, provide the previous output of the model as a prompt for the next window; disabling may make the text inconsistent across windows, but the model becomes less prone to getting stuck in a failure loop.
#@markdown ---
fp16 = True #@param {type:"boolean"}
#@markdown > whether to perform inference in fp16.
#@markdown ---
compression_ratio_threshold = 2.4 #@param {type:"number"}
#@markdown > If the gzip compression ratio is higher than this value, treat the decoding as failed.
#@markdown ---
logprob_threshold = -1.0 #@param {type:"number"}
#@markdown > If the average log probability is lower than this value, treat the decoding as failed.
#@markdown ---
no_speech_threshold = 0.6 #@param {type:"slider", min:-0.0, max:1, step:0.05}
#@markdown > If the probability of the <|nospeech|> token is higher than this value AND the decoding has failed due to `logprob_threshold`, consider the segment as silence.
#@markdown ---

verbose_lut = {
    'Live transcription': True,
    'Progress bar': False,
    'None': None
}

import numpy as np
import warnings
import shutil
from pathlib import Path

args = dict(
    language = (None if language == "Auto detection" else language),
    verbose = verbose_lut[verbose],
    task = task,
    temperature = temperature,
    temperature_increment_on_fallback = temperature_increment_on_fallback,
    best_of = best_of,
    beam_size = beam_size,
    patience=patience,
    length_penalty=(length_penalty if length_penalty>=0.0 else None),
    suppress_tokens=suppress_tokens,
    initial_prompt=(None if not initial_prompt else initial_prompt),
    condition_on_previous_text=condition_on_previous_text,
    fp16=fp16,
    compression_ratio_threshold=compression_ratio_threshold,
    logprob_threshold=logprob_threshold,
    no_speech_threshold=no_speech_threshold
)

temperature = args.pop("temperature")
temperature_increment_on_fallback = args.pop("temperature_increment_on_fallback")
if temperature_increment_on_fallback is not None:
    temperature = tuple(np.arange(temperature, 1.0 + 1e-6, temperature_increment_on_fallback))
else:
    temperature = [temperature]

if Model.endswith(".en") and args["language"] not in {"en", "English"}:
    warnings.warn(f"{Model} is an English-only model but receipted '{args['language']}'; using English instead.")
    args["language"] = "en"

display(Markdown(f"### {filepath}"))

audio_path_local = Path(filepath).resolve()
print("audio local path:", audio_path_local)

transcription = whisper.transcribe(
    whisper_model,
    str(audio_path_local),
    temperature=temperature,
    **args,
)

# Save output
whisper.utils.get_writer(
    output_format=output_format,
    output_dir=audio_path_local.parent
)(
    transcription,
    title
)

try:
    if output_format=="all":
        for ext in ('txt', 'vtt', 'srt', 'tsv', 'json'):
            transcript_file_name = title + "." + ext
            display(Markdown(f"**Transcript file created: {transcript_file_name}**"))
    else:
        transcript_file_name = title + "." + output_format

        display(Markdown(f"**Transcript file created: {transcript_file_name}**"))

except:
    display(Markdown(f"**Transcript file created: {transcript_local_path}**"))



### downloads/108.跟傅盛和王俊煜聊：大模型、产品经理和热门AI应‪用‬.m4a

audio local path: /content/downloads/108.跟傅盛和王俊煜聊：大模型、产品经理和热门AI应‪用‬.m4a
[00:00.000 --> 00:07.720] 大家好 我是潘乱 欢迎来到乱盘书
[00:07.720 --> 00:10.760] 这是一档关注商业科技和互联网的对话节目
[00:10.760 --> 00:14.920] 这一期我们的直播主题是聊大模型和产品经理
[00:14.920 --> 00:18.640] 两位嘉宾是移动互联网时代著名的产品经理
[00:18.640 --> 00:22.560] 猎豹移动的创始人傅胜和豌豆莢的创始人王军玉
[00:22.560 --> 00:26.960] 就是我们聊了包括基于大模型的一些热门产品
[00:26.960 --> 00:30.160] 以及AI时代需要什么样的产品经理等等
[00:30.160 --> 00:33.240] 这次直播是我跟傅胜老师第一次通过
[00:33.240 --> 00:34.880] 就是下面口播一个广告
[00:34.880 --> 00:38.920] 本节目由极客公务员旗下的创始人社区Fundpark发起
[00:38.920 --> 00:41.080] 我跟张鹏老师一起主持
[00:41.080 --> 00:43.920] 如果有AI方向的创业者想加群
[00:43.920 --> 00:46.160] 请搜索关注Fundpark公众号
[00:46.160 --> 00:49.720] 并回复大模型三个字就可以收到报名表单
[00:49.720 --> 00:52.000] OK 我们直接切入正题
[00:52.680 --> 00:56.480] 军玉最近应该是全力投入在研究GDP4
[00:56.520 --> 01:00.960] 就是从你的视角来去看GDP4在这一波相对于3.5
[01:00.960 --> 01:03.440] 它的变化是怎么样的
[01:03.440 --> 01:04.800] 你的感受是怎么样的
[01:04.800 --> 01:05.840] 是不是先说一说
[01:06.440 --> 01:09.840] 其实我这两天一直在玩还没有特明显的感受
[01:09.840 --> 01:10

**Transcript file created: 108.跟傅盛和王俊煜聊：大模型、产品经理和热门AI应‪用‬.srt**

In [32]:
#@markdown # **Download the subtitle file** 🎆

from google.colab import files

output_dir = str(audio_path_local.parent)

if output_format=="all":
    for ext in ('txt', 'vtt', 'srt', 'tsv', 'json'):
        transcript_file_name = audio_path_local.stem + "." + ext
        display(Markdown(f"**Download: {transcript_file_name}**"))
        files.download(output_dir + "/" + transcript_file_name)
else:
    transcript_file_name = audio_path_local.stem + "." + output_format

    display(Markdown(f"**Download: {transcript_file_name}**"))
    files.download(output_dir + "/" + transcript_file_name)


**Download: 108.跟傅盛和王俊煜聊：大模型、产品经理和热门AI应‪用‬.srt**

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>